For predicting Sim to Exp, Black and white

In [ ]:

"""
This displays all of the image files generated from the inference analysis script inference_simtoexp_BnW.py

"""


import os
import re
import matplotlib.pyplot as plt
import cv2
from matplotlib.backends.backend_pdf import PdfPages
from cldm.preprocess import preprocess_simulation,preprocess_experimental,preprocess_experimental_fromscratch



def read_image(filepath):
    """Read image with cv2 and convert BGR to RGB if needed."""
    img = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    if img is None:
        return None
    if len(img.shape) == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img



task_prediction='simtoexp_BnW'
# --- Set folder paths ---

sim_folder = "/hpc/group/youlab/ks723/storage/MATLAB_SIMS/Sim_031524/Final_Test_set"
pred_folder = "/hpc/dctrl/ks723/inference/v2025211_2357_simtoexp_BnW"
gt_folder = "/hpc/group/youlab/ks723/storage/Exp_images/Final_Test_set"

# --- List files ---
sim_files = sorted(os.listdir(sim_folder))
pred_files = sorted(os.listdir(pred_folder))
gt_files = sorted(os.listdir(gt_folder))

# --- Group Simulation (Input) Images ---
sim_pattern = re.compile(r'(\d+)_\d+\.(TIF|tif|png|jpg)$')
sim_dict = {}
for f in sim_files:
    m = sim_pattern.match(f)
    if m:
        cond = m.group(1)
        if cond not in sim_dict:
            sim_dict[cond] = os.path.join(sim_folder, f)

# --- Group Prediction Images (Display ALL predictions) ---
pred_pattern = re.compile(r'(\d+)_([\d]+)_([\d]+)\.(png|tif|TIF|jpg)$')
pred_dict = {}
for f in pred_files:
    m = pred_pattern.match(f)
    if m:
        cond = m.group(1)
        rep = int(m.group(2))
        sample = int(m.group(3))
        path = os.path.join(pred_folder, f)
        pred_dict.setdefault(cond, []).append((rep, sample, path))
# Sort predictions by replicate then sample number.
for cond in pred_dict:
    pred_dict[cond].sort(key=lambda x: (x[0], x[1]))

# --- Group Ground Truth Images ---
gt_pattern = re.compile(r'(\d+)_([\d]+)\.(TIF|tif|png|jpg)$')
gt_dict = {}
for f in gt_files:
    m = gt_pattern.match(f)
    if m:
        cond, rep = m.group(1), int(m.group(2))
        gt_dict.setdefault(cond, []).append((rep, os.path.join(gt_folder, f)))
for cond in gt_dict:
    gt_dict[cond] = sorted(gt_dict[cond], key=lambda x: x[0])

# --- Determine all conditions ---
conds = sorted(set(sim_dict.keys()) | set(pred_dict.keys()) | set(gt_dict.keys()), key=int)

# --- Create PDF ---
pdf_out = f"output_{task_prediction}.pdf"
with PdfPages(pdf_out) as pdf:
    for cond in conds:
        preds = pred_dict.get(cond, [])
        gts = gt_dict.get(cond, [])
        n_pred = len(preds)
        n_gt = len(gts)
        n_rows = max(n_pred, n_gt, 1)

        fig, axes = plt.subplots(n_rows, 3, figsize=(10, n_rows*3))
        if n_rows == 1:
            axes = [axes]

        # Set column titles on the first row
        axes[0][0].set_title("Input")
        axes[0][1].set_title("Predicted Output")
        axes[0][2].set_title("Ground Truth")

        for i in range(n_rows):
            # Column 1: Input (only in first row)
            if i == 0 and cond in sim_dict:
                img = preprocess_simulation(sim_dict[cond])
                axes[i][0].imshow(img, cmap='gray')
            else:
                axes[i][0].axis('off')

            # Column 2: Prediction images
            if i < n_pred:
                _,_, pred_path = preds[i]
                img = read_image(pred_path)
                axes[i][1].imshow(img, cmap='gray')
            else:
                axes[i][1].axis('off')

            # Column 3: Ground Truth images
            if i < n_gt:
                _, gt_path = gts[i]
                img = preprocess_experimental_fromscratch(gt_path)
                axes[i][2].imshow(img, cmap='gray')
            else:
                axes[i][2].axis('off')

            for ax in axes[i]:
                ax.axis('off')

        fig.suptitle(f"Condition {cond}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig)
        plt.close(fig)


In [ ]:
"""
This displays images in a grid format used in current version of paper for Sim to Exp BnW prediction
contains a fixed no of samples from the original prediction

"""

import os
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from matplotlib.backends.backend_pdf import PdfPages
from cldm.preprocess import preprocess_vanillaresize, preprocess_experimental_initialstage

def read_image(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    if img is None:
        return None
    if len(img.shape) == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

task_prediction='simtoexp_BnW'
# --- Set folder paths ---

sim_folder = "/hpc/group/youlab/ks723/storage/MATLAB_SIMS/Sim_031524/Final_Test_set"
pred_folder = "/hpc/dctrl/ks723/inference/v2025211_2357_simtoexp_BnW"
gt_folder = "/hpc/group/youlab/ks723/storage/Exp_images/Final_Test_set"

# --- Group files ---
sim_files = sorted(os.listdir(sim_folder))
pred_files = sorted(os.listdir(pred_folder))
gt_files = sorted(os.listdir(gt_folder))

sim_pattern = re.compile(r'(\d+)_\d+\.(TIF|tif|png|jpg)$')
sim_dict = {m.group(1): os.path.join(sim_folder, f)
            for f in sim_files if (m := sim_pattern.match(f))}

pred_pattern = re.compile(r'(\d+)_([\d]+)_([\d]+)\.(png|tif|TIF|jpg)$')
pred_dict = {}
for f in pred_files:
    if (m := pred_pattern.match(f)):
        cond = m.group(1)
        rep, sample = int(m.group(2)), int(m.group(3))
        pred_dict.setdefault(cond, []).append((rep, sample, os.path.join(pred_folder, f)))
for cond in pred_dict:
    pred_dict[cond].sort(key=lambda x: (x[0], x[1]))

gt_pattern = re.compile(r'(\d+)_([\d]+)\.(TIF|tif|png|jpg)$')
gt_dict = {}
for f in gt_files:
    if (m := gt_pattern.match(f)):
        cond, rep = m.group(1), int(m.group(2))
        gt_dict.setdefault(cond, []).append((rep, os.path.join(gt_folder, f)))
for cond in gt_dict:
    gt_dict[cond] = sorted(gt_dict[cond], key=lambda x: x[0])

conds = sorted(set(sim_dict) | set(pred_dict) | set(gt_dict), key=int)

max_disp = 3  # maximum number to display

pdf_out = f"output_{task_prediction}.pdf"
with PdfPages(pdf_out) as pdf:
    for cond in conds:
        sim_path = sim_dict.get(cond)
        preds = pred_dict.get(cond, [])
        gts = gt_dict.get(cond, [])
        # Show as many images as available in both rows (up to max_disp)
        n_disp = min(max_disp, len(preds), len(gts))
        if n_disp == 0:
            continue

        # Create a 3-row GridSpec that spans nearly the entire figure
        fig = plt.figure(figsize=(n_disp * 3, 9))
        gs = gridspec.GridSpec(3, n_disp, top=0.95, bottom=0.05, hspace=0.2)

        # --- Simulation row (row 0): Center the image with the same size as one cell ---
        if n_disp % 2 == 1:
            ax_sim = fig.add_subplot(gs[0, n_disp // 2])
        else:
            # For even n_disp, compute average center of the two middle cells.
            bbox1 = gs[0, n_disp//2 - 1].get_position(fig)
            bbox2 = gs[0, n_disp//2].get_position(fig)
            center_x = (bbox1.x0 + bbox1.width/2 + bbox2.x0 + bbox2.width/2) / 2
            cell_width = bbox1.width
            cell_height = bbox1.height
            sim_left = center_x - cell_width/2
            sim_bottom = bbox1.y0
            ax_sim = fig.add_axes([sim_left, sim_bottom, cell_width, cell_height])
        if sim_path:
            sim_img = preprocess_vanillaresize(sim_path)
            ax_sim.imshow(sim_img, cmap='gray')
            ax_sim.set_title("Input")
        ax_sim.axis('off')
        # Hide unused axes in row 0
        for j in range(n_disp):
            if (n_disp % 2 == 1 and j != n_disp//2) or (n_disp % 2 == 0 and j in [n_disp//2 - 1, n_disp//2]):
                fig.add_subplot(gs[0, j]).axis('off')

        # --- Prediction row (row 1) ---
        for j in range(n_disp):
            ax = fig.add_subplot(gs[1, j])
            _, _, pred_path = preds[j]
            pred_img = read_image(pred_path)
            ax.imshow(pred_img)
            if j == 0:
                ax.set_ylabel("Predicted Output", fontsize=12)
            ax.axis('off')

        # --- Ground Truth row (row 2) ---
        for j in range(n_disp):
            ax = fig.add_subplot(gs[2, j])
            _, gt_path = gts[j]
            gt_img = preprocess_experimental_fromscratch(gt_path)
            ax.imshow(gt_img, cmap='gray')
            if j == 0:
                ax.set_ylabel("Ground Truth", fontsize=12)
            ax.axis('off')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig)
        plt.close(fig)


/tmp/ipykernel_1431738/4273553075.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.95])


For predicting Sim to Exp, color

In [ ]:

"""
This displays all of the image files generated from the inference analysis script inference_simtoexp.py

"""



import os
import re
import matplotlib.pyplot as plt
import cv2
from matplotlib.backends.backend_pdf import PdfPages
from cldm.preprocess import preprocess_vanillaresize,preprocess_experimental_initialstage,preprocess_experimental_fromscratch



def read_image(filepath):
    """Read image with cv2 and convert BGR to RGB if needed."""
    img = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    if img is None:
        return None
    if len(img.shape) == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img



task_prediction='simtoexpBnW'

# --- Set folder paths ---

sim_folder = "/hpc/group/youlab/ks723/storage/MATLAB_SIMS/Sim_031524/Final_Test_set"
pred_folder = "/hpc/dctrl/ks723/inference/v2025212_00_simtoexp"
gt_folder = "/hpc/group/youlab/ks723/storage/Exp_images/Final_Test_set"

# --- List files ---
sim_files = sorted(os.listdir(sim_folder))
pred_files = sorted(os.listdir(pred_folder))
gt_files = sorted(os.listdir(gt_folder))

# --- Group Simulation (Input) Images ---
sim_pattern = re.compile(r'(\d+)_\d+\.(TIF|tif|png|jpg)$')
sim_dict = {}
for f in sim_files:
    m = sim_pattern.match(f)
    if m:
        cond = m.group(1)
        if cond not in sim_dict:
            sim_dict[cond] = os.path.join(sim_folder, f)

# --- Group Prediction Images (Display ALL predictions) ---
pred_pattern = re.compile(r'(\d+)_([\d]+)_([\d]+)\.(png|tif|TIF|jpg)$')
pred_dict = {}
for f in pred_files:
    m = pred_pattern.match(f)
    if m:
        cond = m.group(1)
        rep = int(m.group(2))
        sample = int(m.group(3))
        path = os.path.join(pred_folder, f)
        pred_dict.setdefault(cond, []).append((rep, sample, path))
# Sort predictions by replicate then sample number.
for cond in pred_dict:
    pred_dict[cond].sort(key=lambda x: (x[0], x[1]))

# --- Group Ground Truth Images ---
gt_pattern = re.compile(r'(\d+)_([\d]+)\.(TIF|tif|png|jpg)$')
gt_dict = {}
for f in gt_files:
    m = gt_pattern.match(f)
    if m:
        cond, rep = m.group(1), int(m.group(2))
        gt_dict.setdefault(cond, []).append((rep, os.path.join(gt_folder, f)))
for cond in gt_dict:
    gt_dict[cond] = sorted(gt_dict[cond], key=lambda x: x[0])

# --- Determine all conditions ---
conds = sorted(set(sim_dict.keys()) | set(pred_dict.keys()) | set(gt_dict.keys()), key=int)

# --- Create PDF ---
pdf_out = f"output_{task_prediction}.pdf"
with PdfPages(pdf_out) as pdf:
    for cond in conds:
        preds = pred_dict.get(cond, [])
        gts = gt_dict.get(cond, [])
        n_pred = len(preds)
        n_gt = len(gts)
        n_rows = max(n_pred, n_gt, 1)

        fig, axes = plt.subplots(n_rows, 3, figsize=(10, n_rows*3))
        if n_rows == 1:
            axes = [axes]

        # Set column titles on the first row
        axes[0][0].set_title("Input")
        axes[0][1].set_title("Predicted Output")
        axes[0][2].set_title("Ground Truth")

        for i in range(n_rows):
            # Column 1: Input (only in first row)
            if i == 0 and cond in sim_dict:
                img = preprocess_vanillaresize(sim_dict[cond])
                axes[i][0].imshow(img, cmap='gray')
            else:
                axes[i][0].axis('off')

            # Column 2: Prediction images
            if i < n_pred:
                _,_, pred_path = preds[i]
                img = read_image(pred_path)
                axes[i][1].imshow(img)
            else:
                axes[i][1].axis('off')

            # Column 3: Ground Truth images
            if i < n_gt:
                _, gt_path = gts[i]
                img = preprocess_experimental_fromscratch(gt_path)
                axes[i][2].imshow(img, cmap='gray')
            else:
                axes[i][2].axis('off')

            for ax in axes[i]:
                ax.axis('off')

        fig.suptitle(f"Condition {cond}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig)
        plt.close(fig)


In [ ]:

"""
This displays images in a grid format used in current version of paper for Sim to Exp color prediction
contains a fixed no of samples from the original prediction

"""

import os
import re
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from matplotlib.backends.backend_pdf import PdfPages
from cldm.preprocess import preprocess_vanillaresize, preprocess_experimental_initialstage

def read_image(filepath):
    img = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    if img is None:
        return None
    if len(img.shape) == 3 and img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

task_prediction = 'simtoexp'

# --- Folder paths ---
sim_folder = "/hpc/group/youlab/ks723/storage/MATLAB_SIMS/Sim_031524/Final_Test_set"
pred_folder = "/hpc/dctrl/ks723/inference/v2025212_00_simtoexp"
gt_folder = "/hpc/group/youlab/ks723/storage/Exp_images/Final_Test_set"

# --- Group files ---
sim_files = sorted(os.listdir(sim_folder))
pred_files = sorted(os.listdir(pred_folder))
gt_files = sorted(os.listdir(gt_folder))

sim_pattern = re.compile(r'(\d+)_\d+\.(TIF|tif|png|jpg)$')
sim_dict = {m.group(1): os.path.join(sim_folder, f)
            for f in sim_files if (m := sim_pattern.match(f))}

pred_pattern = re.compile(r'(\d+)_([\d]+)_([\d]+)\.(png|tif|TIF|jpg)$')
pred_dict = {}
for f in pred_files:
    if (m := pred_pattern.match(f)):
        cond = m.group(1)
        rep, sample = int(m.group(2)), int(m.group(3))
        pred_dict.setdefault(cond, []).append((rep, sample, os.path.join(pred_folder, f)))
for cond in pred_dict:
    pred_dict[cond].sort(key=lambda x: (x[0], x[1]))

gt_pattern = re.compile(r'(\d+)_([\d]+)\.(TIF|tif|png|jpg)$')
gt_dict = {}
for f in gt_files:
    if (m := gt_pattern.match(f)):
        cond, rep = m.group(1), int(m.group(2))
        gt_dict.setdefault(cond, []).append((rep, os.path.join(gt_folder, f)))
for cond in gt_dict:
    gt_dict[cond] = sorted(gt_dict[cond], key=lambda x: x[0])

conds = sorted(set(sim_dict) | set(pred_dict) | set(gt_dict), key=int)

max_disp = 3  # maximum number to display

pdf_out = f"output_{task_prediction}.pdf"
with PdfPages(pdf_out) as pdf:
    for cond in conds:
        sim_path = sim_dict.get(cond)
        preds = pred_dict.get(cond, [])
        gts = gt_dict.get(cond, [])
        # Show as many images as available in both rows (up to max_disp)
        n_disp = min(max_disp, len(preds), len(gts))
        if n_disp == 0:
            continue

        # Create a 3-row GridSpec that spans nearly the entire figure
        fig = plt.figure(figsize=(n_disp * 3, 9))
        gs = gridspec.GridSpec(3, n_disp, top=0.95, bottom=0.05, hspace=0.2)

        # --- Simulation row (row 0): Center the image with the same size as one cell ---
        if n_disp % 2 == 1:
            ax_sim = fig.add_subplot(gs[0, n_disp // 2])
        else:
            # For even n_disp, compute average center of the two middle cells.
            bbox1 = gs[0, n_disp//2 - 1].get_position(fig)
            bbox2 = gs[0, n_disp//2].get_position(fig)
            center_x = (bbox1.x0 + bbox1.width/2 + bbox2.x0 + bbox2.width/2) / 2
            cell_width = bbox1.width
            cell_height = bbox1.height
            sim_left = center_x - cell_width/2
            sim_bottom = bbox1.y0
            ax_sim = fig.add_axes([sim_left, sim_bottom, cell_width, cell_height])
        if sim_path:
            sim_img = preprocess_vanillaresize(sim_path)
            ax_sim.imshow(sim_img, cmap='gray')
            ax_sim.set_title("Input")
        ax_sim.axis('off')
        # Hide unused axes in row 0
        for j in range(n_disp):
            if (n_disp % 2 == 1 and j != n_disp//2) or (n_disp % 2 == 0 and j in [n_disp//2 - 1, n_disp//2]):
                fig.add_subplot(gs[0, j]).axis('off')

        # --- Prediction row (row 1) ---
        for j in range(n_disp):
            ax = fig.add_subplot(gs[1, j])
            _, _, pred_path = preds[j]
            pred_img = read_image(pred_path)
            ax.imshow(pred_img)
            if j == 0:
                ax.set_ylabel("Predicted Output", fontsize=12)
            ax.axis('off')

        # --- Ground Truth row (row 2) ---
        for j in range(n_disp):
            ax = fig.add_subplot(gs[2, j])
            _, gt_path = gts[j]
            gt_img = preprocess_experimental_initialstage(gt_path)
            ax.imshow(gt_img, cmap='gray')
            if j == 0:
                ax.set_ylabel("Ground Truth", fontsize=12)
            ax.axis('off')

        plt.tight_layout(rect=[0, 0, 1, 0.95])
        pdf.savefig(fig)
        plt.close(fig)


/tmp/ipykernel_1431738/2682329500.py:114: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 1, 0.95])
